In [1]:
# Importing the required libraries 
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from pyculiarity import detect_ts

# Load Meter and Meta Data

In [2]:
all_meters = pd.read_csv("../../data/meters/cleaned/allmeters_daily.csv")
meta_data = pd.read_csv("../../data/metadata/metadata.csv")

We are only interested looking at Energy Consumption by Educational Institutions and hence using the primaryspaceusage to filter for Educational Institions

In [3]:
meta_education = meta_data[meta_data['primaryspaceusage'] == 'Education']
meta_education

,building_id,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,lat,lng,...,yearbuilt,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating
3,Panther_education_Rosalie,Panther,0.0,0.0,Education,Research,690.5,7432.0,28.517689,-81.379039,...,2008.0,NaN,NaN,NaN,NaN,276,NaN,NaN,NaN,NaN
4,Panther_education_Misty,Panther,1.0,0.0,Education,Research,252.7,2720.0,28.517689,-81.379039,...,2004.0,NaN,NaN,NaN,NaN,375,NaN,NaN,NaN,NaN
6,Panther_education_Mattie,Panther,2.0,0.0,Education,Classroom,499.4,5376.0,28.517689,-81.379039,...,1991.0,NaN,NaN,NaN,NaN,49,NaN,NaN,NaN,NaN
8,Panther_education_Diann,Panther,3.0,0.0,Education,Classroom,2200.4,23685.0,28.517689,-81.379039,...,2002.0,NaN,NaN,NaN,NaN,145,NaN,NaN,NaN,NaN
9,Panther_education_Gina,Panther,4.0,0.0,Education,Research,10833.1,116607.0,28.517689,-81.379039,...,1975.0,NaN,NaN,NaN,NaN,546,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618,Cockatoo_education_Marva,Cockatoo,1438.0,15.0,Education,College Classroom,10123.7,108971.0,42.459837,-76.485292,...,1990.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1619,Cockatoo_education_Collin,Cockatoo,1439.0,15.0,Education,College Laboratory,5248.7,56497.0,42.459837,-76.485292,...,1957.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1621,Cockatoo_education_Gussie,Cockatoo,1441.0,15.0,Education,College Laboratory,2800.4,30143.0,42.459837,-76.485292,...,1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1623,Cockatoo_education_Arlen,Cockatoo,1443.0,15.0,Education,College Classroom,3745.0,40311.0,42.459837,-76.485292,...,1913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have 617 buildings that were used for Educational purposes

In [4]:
# Since not all buildings use all energy sources, marking the sources used for each building 
meter_columns = ['electricity', 'hotwater', 'chilledwater', 'steam', 'water', 'irrigation', 'solar', 'gas']
for col in meter_columns:
    meta_education[col] = np.where(meta_education[col] == 'Yes', 1, 0)  

/var/folders/46/4q6_331j7qx5_bf3yg_mv9km0000gn/T/ipykernel_16950/597666801.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_education[col] = np.where(meta_education[col] == 'Yes', 1, 0)


In [5]:
# Melt the DataFrame to pivot it
meta_melted = meta_education.melt(id_vars=[col for col in meta_education.columns if col not in meter_columns], 
                    value_vars=meter_columns, 
                    var_name='meter', 
                    value_name='value')

# Filtering for the corrrect 
meta_melted = meta_melted[meta_melted['value']==1]

In [6]:
# Merging both the dataframes together
merged_df = pd.merge(all_meters, meta_melted, on=['building_id','meter' ], how='inner')

In [7]:
merged_df.columns

Index(['building_id', 'meter', 'date', 'meter_reading', 'site_id',
       'building_id_kaggle', 'site_id_kaggle', 'primaryspaceusage',
       'sub_primaryspaceusage', 'sqm', 'sqft', 'lat', 'lng', 'timezone',
       'industry', 'subindustry', 'heatingtype', 'yearbuilt', 'date_opened',
       'numberoffloors', 'occupants', 'energystarscore', 'eui', 'site_eui',
       'source_eui', 'leed_level', 'rating', 'value'],
      dtype='object')

In [8]:
merged_df.head()

,building_id,meter,date,meter_reading,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,...,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating,value
0,Bear_education_Alfredo,electricity,2016-01-01,2.9050,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Bear_education_Alfredo,electricity,2016-01-02,2.7700,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Bear_education_Alfredo,electricity,2016-01-03,2.6725,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Bear_education_Alfredo,electricity,2016-01-04,4.5650,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Bear_education_Alfredo,electricity,2016-01-05,4.7825,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [9]:
# Inspecting the dataframe for a single building id 
merged_df[merged_df['building_id_kaggle'] == 636]

,building_id,meter,date,meter_reading,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,...,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating,value
0,Bear_education_Alfredo,electricity,2016-01-01,2.9050,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Bear_education_Alfredo,electricity,2016-01-02,2.7700,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Bear_education_Alfredo,electricity,2016-01-03,2.6725,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Bear_education_Alfredo,electricity,2016-01-04,4.5650,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Bear_education_Alfredo,electricity,2016-01-05,4.7825,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,Bear_education_Alfredo,electricity,2017-12-27,3.9825,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
727,Bear_education_Alfredo,electricity,2017-12-28,3.9975,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
728,Bear_education_Alfredo,electricity,2017-12-29,3.3650,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
729,Bear_education_Alfredo,electricity,2017-12-30,3.1825,Bear,636.0,4.0,Education,Education,609.8,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


# Missing Data

In [ ]:
def determine_column_counts(df):
    columns = df.columns
    data_counts = pd.DataFrame(columns = ['column_name','column_type','num_rows','distinct_data', 'distinct_percent', 'missing_data', 'missing_percentage'])
    
    #For each column, calculate number and percentage of missing rows and distinct counts
    for col in columns:
        col_vals = df[col]
        total_data = len(col_vals)
        col_missing = sum(col_vals.isnull())
        missing_percent = round((col_missing / len(df[col])) * 100 , 3)
        col_distinct = (col_vals.nunique())
        distinct_percent = round((col_distinct / len(df[col])) * 100 , 3)

        data_counts.loc[len(data_counts)] = [col, col_vals.dtype, total_data, col_distinct, distinct_percent, col_missing, missing_percent]
        
    return(data_counts)

missing_data = determine_column_counts(merged_df)
missing_data

On the basis of missing values and domain knowledge we shall drop the following columns since they are not directly consequential to our analysis

In [ ]:
cleaned_df = merged_df.drop(columns = ['primaryspaceusage', 'lat', 'lng','industry', 'subindustry',
                                      'heatingtype', 'yearbuilt', 'date_opened', 'numberoffloors', 'occupants', 'energystarscore', 
                                      'eui', 'site_eui', 'source_eui','leed_level', 'rating', 'value' ])

In [ ]:
cleaned_df.to_csv("../data/cleaned/cleaned_meters_meta.csv")

In [ ]:
cleaned_df